## A Simple Model

In [2]:
from __future__ import print_function
import numpy as np
data_path = "/home/ubuntu/data/"

### Training

Training Dataset (train and val)

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_image_dim_ordering('th')

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5103)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [18]:
batch_size = 80
img_size=(320, 180)

In [19]:
train_datagen = ImageDataGenerator(rescale=1./255, 
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        data_path+"train/"+'1trainDir',
        target_size=img_size, 
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=True)

validation_generator = val_datagen.flow_from_directory(
        data_path+"train/"+'1valDir',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

train_n = train_generator.N
val_n = validation_generator.N

Found 3020 images belonging to 8 classes.
Found 376 images belonging to 8 classes.


In [6]:
train_generator.class_indices

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

Model Definition

In [7]:
nb_classes = len(train_generator.class_indices)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

In [20]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_size[0], img_size[1]), border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.333))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.333))

model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.333))

model.add(Convolution2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.333))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

Model Training

In [22]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0)
checkpoints = ModelCheckpoint(filepath=data_path+"best_weights.hdf5", verbose=0, save_best_only=True)
callsbacks = [early_stopping, checkpoints]

In [23]:
model.fit_generator(
        train_generator,
        samples_per_epoch=train_n,
        nb_epoch=75,
        validation_data=validation_generator,
        nb_val_samples=val_n,
        callbacks=callsbacks, 
        verbose=1)

Epoch 1/75
3020/3020 [==============================] - 69s - loss: 1.8506 - acc: 0.3497 - val_loss: 1.9704 - val_acc: 0.4202
Epoch 2/75
3020/3020 [==============================] - 62s - loss: 1.7574 - acc: 0.4159 - val_loss: 1.9766 - val_acc: 0.4282
Epoch 3/75
3020/3020 [==============================] - 62s - loss: 1.7111 - acc: 0.4295 - val_loss: 1.9046 - val_acc: 0.4149
Epoch 4/75
3020/3020 [==============================] - 61s - loss: 1.6952 - acc: 0.4414 - val_loss: 1.9325 - val_acc: 0.4335
Epoch 5/75
3020/3020 [==============================] - 62s - loss: 1.6887 - acc: 0.4437 - val_loss: 1.8870 - val_acc: 0.4468
Epoch 6/75
3020/3020 [==============================] - 61s - loss: 1.6762 - acc: 0.4467 - val_loss: 1.8224 - val_acc: 0.4202
Epoch 7/75
3020/3020 [==============================] - 62s - loss: 1.6612 - acc: 0.4450 - val_loss: 1.8529 - val_acc: 0.4202
Epoch 8/75
3020/3020 [==============================] - 62s - loss: 1.6510 - acc: 0.4487 - val_loss: 1.8437 - val_acc:

Loading Best Model

In [25]:
from keras.models import load_model
model = load_model(data_path+"best_weights.hdf5")

Model Testing

In [28]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        data_path+'train/'+'1testDir',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
test_n = test_generator.N

Found 381 images belonging to 8 classes.


In [29]:
print("Model Loss, Accuracy: ")
print(model.evaluate_generator(test_generator, val_samples=test_n))

Model Loss, Accuracy: 
[0.91526978300625261, 0.66929133600137369]


In [30]:
classes = {v: k for k, v in train_generator.class_indices.iteritems()}
print(classes)

{0: 'ALB', 1: 'BET', 2: 'DOL', 3: 'LAG', 4: 'NoF', 5: 'OTHER', 6: 'SHARK', 7: 'YFT'}


Confusion Matrix

In [31]:
from sklearn.metrics import classification_report, confusion_matrix

In [32]:
Y_test = []
Y_preds = []
batch_ind = test_generator.batch_index
for X_batch, Y_batch in test_generator:
    y_pred = model.predict_classes(X_batch, verbose=0)
    Y_test.extend(np.argmax(Y_batch,axis=1))
    Y_preds.extend(y_pred)
    if test_generator.batch_index == batch_ind:
        break

In [33]:
names = range(len(classes.keys()))
for k,v in classes.iteritems():
    names[k] = v

In [34]:
print(classification_report(Y_test, Y_preds, target_names=names))

             precision    recall  f1-score   support

        ALB       0.65      0.96      0.78       180
        BET       0.00      0.00      0.00        19
        DOL       0.00      0.00      0.00        13
        LAG       0.00      0.00      0.00         8
        NoF       0.89      0.75      0.81        44
      OTHER       0.00      0.00      0.00        24
      SHARK       0.82      0.64      0.72        14
        YFT       0.60      0.51      0.55        79

avg / total       0.56      0.67      0.60       381



/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:
print(confusion_matrix(Y_test, Y_preds))

[[173   0   0   0   3   0   0   4]
 [ 14   0   0   0   0   0   2   3]
 [  2   0   0   0   0   0   0  11]
 [  6   0   0   0   1   0   0   1]
 [  6   0   0   0  33   0   0   5]
 [ 22   0   0   0   0   0   0   2]
 [  4   0   0   0   0   0   9   1]
 [ 39   0   0   0   0   0   0  40]]


Saving Trained Model

In [36]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open(data_path+"Simple_Keras-model2.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
    
# serialize weights to HDF5
model.save_weights(data_path+'Simple_Keras-model2.h5') 

# serialize class indices to a numpy file npy
np.save('/home/ubuntu/data/Simple_Keras-model2-classes.npy', classes)

### Predicting

Generating predictions on the test set

In [37]:
from keras.preprocessing.image import img_to_array, load_img
def get_image_as_X(path_to_image_file, target_size, dim_ordering='th', rescale=1./255):
    img = load_img(path_to_image_file, grayscale=False, target_size=target_size)
    x = img_to_array(img, dim_ordering=dim_ordering)
    #x = image_data_generator.standardize(x)
    x *= rescale
    return np.array([x])

In [38]:
import pandas as pd

In [44]:
import os
test_path = data_path + "test_stg1/"
imaglist = []
for (dirpath, dirnames, filenames) in os.walk(test_path):
    imaglist.extend(filenames)
    break
predictions = []
for element in imaglist:
    x = get_image_as_X(test_path + "/" + element, img_size)
    p = model.predict_proba(x, batch_size=1, verbose=0)
    predictions.extend(p)
test_id = imaglist[:]

In [45]:
import datetime
def create_submission(path, predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    result1.loc[:, 'image'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print("Writing " + sub_file)
    result1.to_csv(path+sub_file, index=False)

In [47]:
create_submission(data_path, predictions, test_id, "Simple_Keras_2")

Writing submission_Simple_Keras_2_2016-12-12-04-02.csv


Public Score: 1.25864 (multi-class logarithmic loss.)